##### Compare our code with Dr. Adams solution

In [1]:
# import pickle file and print the content
import pickle

# variableAndQmatrix.pickle
with open('variableAndQmatrix.pickle', 'rb') as f:
    data = pickle.load(f)


In [2]:
import numpy as np
# get the determinant of each Q matrix
for variable, Qmatrix in data.items():
    print(variable, sep='\n')
    print('Determinant:', (np.linalg.det(Qmatrix), 2))


EP_DISABL
Determinant: (array([0.]), 2)
EP_PCI
Determinant: (array([0.]), 2)
EP_LIMENG
Determinant: (array([-1.17615891e-10]), 2)
EP_CROWD
Determinant: (array([0.]), 2)
EP_UNINSUR
Determinant: (array([3.91085014e-15]), 2)


In [3]:
Qmatrix = data['EP_CROWD'][0]

In [73]:
# ger the shape of the df
print(Qmatrix.shape)

(11, 11)


In [76]:
type(Qmatrix)

pandas.core.frame.DataFrame

In [83]:
# save all the matrices to different files
for variable, Qmatrix in data.items():
    Qmatrix[0].to_csv('Qmatrix_' + variable+'.csv', index=False)

##### Our code

Compute the generalized inverse \
The generalized inverse of a matrix, also known as the Moore-Penrose inverse (oseudo Inverse)

In [9]:
import numpy as np

In [37]:
# Compute SVD of Q
U, S, Vt = np.linalg.svd(Qmatrix)

In [38]:
# get the machine epsilon
eps = np.finfo(float).eps
tol = np.sqrt(eps)

In [39]:
# Calculate inverse of singular values
S_inv = np.where(S > tol, 1/S, 0)

In [40]:
# Construct the generalized inverse
Q_inv = Vt.T @ np.diag(S_inv) @ U.T

In [41]:
print("Generalized Inverse of Q:")
print(Q_inv)

Generalized Inverse of Q:
[[ 6.40625000e-01 -4.72127334e-16  2.65625000e-01 -4.84375000e-01
  -3.98925408e-16 -2.34375000e-01  3.90625000e-01 -3.67062104e-16
  -2.34375000e-01  1.56250000e-02 -3.59375000e-01]
 [ 1.03208092e-17  5.55555556e-01 -5.97719507e-18  2.46547143e-16
  -4.44444444e-01  2.89774887e-16 -1.30073003e-16 -1.11111111e-01
   2.24169183e-16  1.18118592e-16  2.38399518e-16]
 [ 2.65625000e-01 -1.14673678e-15  1.89062500e+00 -8.59375000e-01
  -1.55295014e-15 -6.09375000e-01  1.01562500e+00 -1.14605996e-15
  -6.09375000e-01 -3.59375000e-01 -7.34375000e-01]
 [-4.84375000e-01  9.56117452e-16 -8.59375000e-01  1.39062500e+00
   9.51662720e-16  1.40625000e-01 -7.34375000e-01  6.92122770e-16
   1.40625000e-01 -1.09375000e-01  5.15625000e-01]
 [-1.73150739e-16 -4.44444444e-01 -7.31692140e-16  7.56367061e-17
   5.55555556e-01  4.45551972e-17 -3.78198815e-16 -1.11111111e-01
   3.36211767e-17  4.16595845e-17  9.47955579e-17]
 [-2.34375000e-01  3.96490134e-16 -6.09375000e-01  1.406250

Compute generalized variance - using Q Inv

In [42]:
generalized_variance = np.exp(np.mean(np.log(np.diag(Q_inv))))  # equation in the paper use daba as 1
generalized_variance #fac in R code

0.6501074433163458

##### Dr. Adams code

In [10]:
import scipy as sp
from scipy.linalg import solve
from scipy.sparse.linalg import spsolve

In [5]:
# Qmatrix dataframe to numpy array
Qmatrix_ = Qmatrix.to_numpy()

In [6]:
n = Qmatrix_.shape[0]
n

12

In [12]:
# Add jitter to the diagonal of 
# In the context of the provided code, "jittered" refers to the process of adding 
# small random variations to the diagonal elements of the matrix Qmatrix_. 
# This is achieved by multiplying a sparse diagonal matrix (constructed with ones along the main diagonal)
# by the maximum value along the diagonal of Qmatrix_ and the square root of the machine epsilon.

Q_jitter = Qmatrix_ + sp.sparse.diags(np.ones(n)) * max(Qmatrix_.diagonal()) * np.sqrt(

    np.finfo(np.float64).eps

)

In [13]:
# Q_jitter = Qmatrix_

In [14]:
# inverse of precision (Q) is cov

Q_perturbed = sp.sparse.csc_array(Q_jitter)

b = sp.sparse.identity(n, format='csc')

sigma = spsolve(Q_perturbed, b)

In [15]:
cov = sigma.toarray()
Q_jitter.dot(cov)

matrix([[ 9.99999999e-01,  0.00000000e+00,  9.31322575e-10,
          0.00000000e+00,  0.00000000e+00, -1.39698386e-09,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         -4.65661287e-10, -9.31322575e-10,  0.00000000e+00,
          4.65661287e-10,  0.00000000e+00, -9.31322575e-10,
         -9.31322575e-10,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  9.99999999e-01,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00, -4.65661287e-10,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00,  4.65661287e-10,  0.00000000e+00,
          4.65661287e-10,  0.00000000e+00,  0.00000000e+00,
          4.65661287e-10,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.0000000

In [16]:
# V \in Null(Q)

V = np.ones(n)  # from pg. 6

W = sigma @ V.T  # \Sigma * B in 3.17

Q_inv = sigma - np.outer(W * solve(V @ W, np.ones(1)), W.T)

In [17]:
Q_inv

array([[ 2796202.85272718, -1398101.33241468,  2796202.60272719,
        -1398101.33241468, -1398101.33241468,  2796202.60272719,
        -1398101.33241468, -1398101.33321033, -1398101.33241468,
        -1398101.33241468,  2796202.60272719, -1398101.33321033],
       [-1398101.33241468,  1398101.65674915, -1398101.33241468,
         1398101.37897139,  1398101.10119363, -1398101.33241468,
         1398101.49008248, -1398101.33319592,  1398101.26786029,
         1398101.10119363, -1398101.33241468, -1398101.33319592],
       [ 2796202.60272719, -1398101.33241468,  2796202.97772718,
        -1398101.33241468, -1398101.33241468,  2796202.60272719,
        -1398101.33241468, -1398101.33321033, -1398101.33241468,
        -1398101.33241468,  2796202.47772719, -1398101.33321033],
       [-1398101.33241468,  1398101.37897139, -1398101.33241468,
         1398101.76786026,  1398101.15674918, -1398101.33241468,
         1398101.21230473, -1398101.33319592,  1398101.32341584,
         1398101.15674

In [18]:
# grabbing diag of cov gives var and

# arithmetic mean in log-space becomes geometric mean after exp

scaling = np.exp(np.sum(np.log(np.diag(Q_inv))) / n)

print(scaling)

2303440.91602172
